In [95]:
import json
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Kkma

# Our Analysis Variables:

- Length of the string 텍스트 스트링의 길이 (for equal comparison)
- Number of tokens 토큰 수
- Number of sentences 문장 수
- Tokens sorted by frequency 빈도 수로 정렬한 토큰 리스트
- Sentence structure category sorted by frequency 빈도 수로 정렬한 문장 구조 카테고리

In [348]:
import json
import re
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm


def get_string_length(text: str) -> int:
    return(len(text))
def get_num_tokens(tokens: list) -> int:
    return(len(tokens))
def find_sent_id(tokens: list) -> list:
    ids = []
    for i, tok in enumerate(tokens):
        found = re.findall('[.?!]+', tok)
        if(bool(found)):
            ids.append(i)
    return ids
def split_sentence(sent_ids: list, tokens: list)-> list:  
    sentences = []
    if bool(sent_ids):
        start = 0
        for i in sent_ids:
            sentences.append(tokens[start:i+1])
            start = i+1
    else:
        sentences.append(tokens)
    return sentences
def split_pos_sentence(sent_ids: list, pos: list)-> list:  
    sentences = []
    if bool(sent_ids):
        start = 0
        for i in sent_ids:
            sent_both = pos[start:i+1] #get the sentence
            sent_tag = list(map(lambda tag: tag[1], sent_both))#get only the pos tags
            sentences.append(sent_tag)#append
            start = i+1
    else:
        sent_both = pos
        sent_tag = list(map(lambda tag: tag[1], sent_both))#get only the pos tags
        sentences.append(sent_tag)
    return sentences
def get_num_sentences(tokens:list, pos:list):
    sent_ids = find_sent_id(tokens)
    sentences = split_sentence(sent_ids, tokens)
    pos_sentences = split_pos_sentence(sent_ids, pos)
    return (len(sent_ids), sentences, pos_sentences)
def get_num_token_sentences(sentences):
    num_tokens = []
    for sentence in sentences:
        num_tokens.append(len(sentence))
    return num_tokens
def simple_analysis(data):
    data['sentence_tokens'] = []
    data['sentence_pos'] = []
    all_analysis = {'string_length':[], 'num_tokens':[], 'num_sentences':[], 'num_token_sentences':[],}
    for i in tqdm(range(len(data['text']))):
        text = data['text'][i]
        tokens = data['tokens'][i]
        pos = data['pos'][i]
        all_analysis['string_length'].append(get_string_length(text))
        all_analysis['num_tokens'].append(get_num_tokens(tokens))
        num_sentences, sentences, pos_sentences = get_num_sentences(tokens, pos)
        num_token_sentences = get_num_token_sentences(sentences)
        all_analysis['num_sentences'].append(num_sentences)
        all_analysis['num_token_sentences'].append(num_token_sentences)
        data['sentence_tokens'].append(sentences)
        data['sentence_pos'].append(pos_sentences)
    
    return data, all_analysis
def get_rel_freq(freq: dict):
    freq = pd.Series(freq, dtype = "float64")
    tot = freq.sum()
    return (freq/tot).to_dict()
def get_pos_freq(articles: list[list]) -> dict:
    pos_freq = {}
    pos_freq['s_freq'] = []
    pos_freq['a_freq'] = []
    pos_freq['s_rel_freq'] = []
    pos_freq['a_rel_freq'] = []
    for i in tqdm(range(len(articles))):
        s_freq = []
        s_rel_freq = []
        article_freq = dict()
        for sentence in articles[i]:
            sentence_freq = dict(Counter(sentence))
            s_freq.append(sentence_freq)
            s_rel_freq.append(get_rel_freq(sentence_freq))
            for (key, freq) in sentence_freq.items():
                if key in article_freq.keys():
                    article_freq[key] = article_freq[key] + freq
                else:
                    article_freq[key] = freq
        pos_freq['s_freq'].append(s_freq)
        pos_freq['s_rel_freq'].append(s_rel_freq)
        pos_freq['a_freq'].append(article_freq)
        pos_freq['a_rel_freq'].append(get_rel_freq(article_freq))
        
    return pos_freq

def calRelPosition(sentence_pos, num_token_sentences):

    for i, sentence in enumerate(sentence_pos):
        pos = np.array(sentence)
        result = {}
        for tag in set(pos):
            indices = np.where(pos == tag)
            result[tag] = (indices/num_token_sentences[i]).tolist()
           
    return result

def get_more_features(data, all_analysis):
    all_analysis['token_variety'] = []
    all_analysis['rel_position'] = []
    for i in tqdm(range(len(data['pos']))):
        num_token_sentences = np.array(all_analysis['num_token_sentences'][i])
        num_token_type = np.array(list(map(len, all_analysis['pos_freq']['s_freq'][i])))
        token_variety = (num_token_type / num_token_sentences).tolist()
        all_analysis['token_variety'].append(token_variety)
        sentence_pos = data['sentence_pos'][i]
        rel_position = calRelPosition(sentence_pos, num_token_sentences)
        all_analysis['rel_position'].append(rel_position)
    return all_analysis


def analysis(data):
    data, all_analysis = simple_analysis(data)
    all_analysis['pos_freq'] = get_pos_freq(data['sentence_pos'])
    # all_analysis = get_more_features(data, all_analysis)
    return data, all_analysis
    

In [4]:
"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/articles_ytn_tokenized.json"
"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/articles_ytn_analyzed.json"

'C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/articles_ytn_analyzed.json'

In [245]:
a = data['sentence_pos'][1215]
b = np.array(new_analysis['num_token_sentences'][1215])
print(len(b))
# calRelPosition(a, b)

0


In [339]:
category = "articles"
data_path = {
    "articles":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/articles_ytn_tokenized.json",
    "abstract":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/abstract_tokenized.json",
    "essay":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/essay_tokenized.json",
    "literature":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/literature_tokenized.json",
    }

with open(data_path[category]) as json_file:
    data = json.load(json_file)

# analysis = {
#     "articles": analysis,
#     "abstract": analysis,
#     "essay": analysis,
#     "literature":analysis,
#     }

# method = analysis[category]
# data, all_analysis = method(data)

# out_path = {
#     "articles":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/analyzed_data/article/articles_ytn_analysis.json",
#     "abstract":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/analyzed_data/abstract/abstract_analysis.json",
#     "essay":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/analyzed_data/essay/essay_analysis.json",
#     "literature":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/analyzed_data/literature/literature_analysis.json",
# }
# data_out_path = {
#     "articles":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/analyzed_data/article/articles_data.json",
#     "abstract":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/analyzed_data/abstract/abstract_data.json",
#     "essay":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/analyzed_data/essay/essay_data.json",
#     "literature":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/analyzed_data/literature/literature_data.json",
# }
# with open(out_path[category], "w") as outfile:
#     json.dump(all_analysis, outfile)
# with open(data_out_path[category], "w") as outfile:
#     json.dump(data, outfile)


In [252]:
# with open(data_path[category], "w") as outfile:
#     json.dump(data, outfile)

In [270]:
def drop_index(index_num, category):
    data_path = {
        "articles":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/articles_ytn_tokenized.json",
        "abstract":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/abstract_tokenized.json",
        "essay":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/essay_tokenized.json",
        "literature":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/literature_tokenized.json",
        }

    with open(data_path[category]) as json_file:
        data = json.load(json_file)

    data['text'].pop(index_num)
    data['tokens'].pop(index_num)
    data['pos'].pop(index_num)
    
    with open(data_path[category], "w") as outfile:
        json.dump(data, outfile)

In [330]:
drop_index(3000+1713, "articles")

In [352]:
category = "articles"
data_path = {
    "articles":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/articles_ytn_tokenized.json",
    "abstract":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/abstract_tokenized.json",
    "essay":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/essay_tokenized.json",
    "literature":"C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/tokenized_data/literature_tokenized.json",
    }

with open(data_path[category]) as json_file:
    data = json.load(json_file)
    
data['text'][5477]

"온라인에서 화제가 된 소식을 전해드리는 '오늘 세 컷'튀르키예·시리아 강진 일주일째를 맞아 피해 현장에는 세계 각국에서 온 구조팀이 사투를 벌이고 있는데요.구조팀에 소속돼 임무를 수행하던 구조견들의 부상과 사망소식도 전해지고 있습니다.앞발에 붕대를 감고 잔해가 깔린 바닥에 앉아있는 구조견.우리 긴급구조대와 함께 튀르키예에 파견된 6살 토백이입니다.며칠 전 구조작업 도중 날카로운 물체에 찔려 다쳤지만, 응급처치를 받고 다시 현장에 투입됐습니다.짠하면서, 대견한 마음도 드는데요.우리 구조대는, 붕대 투혼 중인 토백이를 위해, 위험한 곳에선 직접 안아서 옮겨주며 구조작업을 진행하고 있다고 합니다.세상을 떠난 구조견도 있습니다.멕시코 구조견 프로테오는 현장에서 구조 활동 중 세상을 떠나 애도의 물결이 이어지기도 했는데요.누리꾼들은 이들의 활약에 고마움을 전하며 모두 건강히 돌아오길 기원했습니다.지난해 말 달로 떠난, 달 궤도선 다누리가 처음으로 달 가까이에서 촬영한 사진을 보내왔습니다.화면으로 함께 보시죠.달 표면에서, 웅덩이처럼 유독 크게 패인 이곳은 달에서 가장 큰 바다인, 폭풍의 바다인데요.면적이 한반도 18배 크기에 달합니다.지난 1966년 세계 첫 달 착륙선인 옛소련의 루나 9호가 착륙한 곳이기도 하죠.또, 여러 개의 크레이터가 모여 형성된 레이타 계곡과, 인류 최초로 달 표면 탐사가 이뤄진 '비의 바다'도 생생하게 담겼습니다.다누리는 이 밖에도, 달에서 본 지구의 신비롭고 다양한 모습도 함께 보내왔습니다.시운전을 마치고, 지난 4일부터 정상 운영에 착수한 다누리는, 올해 말까지 달 과학연구과 우주 인터넷 기술 검증 등 임무를 수행할 예정입니다.매년 아동학대 사건이 끊이지 않는데요.피해 아동 대부분은 원래 가정으로 다시 돌아가는 거로 나타났습니다.보건복지부의 '2021년 아동학대 주요통계'를 보면, 부모가 학대 가해자로 나타난 경우가 전체 아동학대 사건의 83.7%로 대부분이었고, 학대 장소도 대부분 가정이었는데요.정작 피해 아동 10명 중 8~9명은 당국

In [341]:
test_data = {
    'text': data['text'][3000:],
    'tokens': data['tokens'][3000:],
    'pos': data['pos'][3000:]
}


In [320]:
index = 1713
test_data['text'].pop(index)
test_data['tokens'].pop(index)
test_data['pos'].pop(index)

[["'", 'SS'],
 ['연기인', 'NNG'],
 ['생', 'XSN'],
 ['67', 'NR'],
 ['년', 'NNM'],
 ['차', 'NNG'],
 ['대배우', 'NNG'],
 ["'", 'SS'],
 ['이순', 'NNG'],
 ['재', 'NNG'],
 ['드라마', 'NNG'],
 ['부터', 'JX'],
 ['영화', 'NNG'],
 [',', 'SP'],
 ['광고', 'NNG'],
 ['까지', 'JX'],
 ['모두', 'MAG'],
 ['섭렵', 'NNG'],
 ['이순', 'NNG'],
 ['재', 'NNG'],
 ['"', 'SS'],
 ['커피', 'NNG'],
 ['광고', 'NNG'],
 ['내가', 'NNG'],
 ['원조', 'NNG'],
 ['…', 'SE'],
 ['나', 'NP'],
 ['다음', 'NNG'],
 ['이', 'JKS'],
 ['안', 'MAG'],
 ['성기', 'NNG'],
 ['"', 'SS'],
 ["'", 'SS'],
 ['거침없이', 'MAG'],
 ['하이', 'NNG'],
 ['킥', 'NNG'],
 ["'", 'SS'],
 ['에서', 'JKM'],
 ['파격적', 'NNG'],
 ['시트콤', 'NNG'],
 ['연기', 'NNG'],
 ['로', 'JKM'],
 ['대중적', 'NNG'],
 ['인기', 'NNG'],
 ['얻', 'VV'],
 ['어', 'ECD'],
 ['구', 'NNG'],
 ['순', 'NNG'],
 ['앞두', 'VV'],
 ['고', 'ECE'],
 ['연극', 'NNG'],
 ["'", 'SS'],
 ['갈매기', 'NNG'],
 ["'", 'SS'],
 ['연출', 'NNG'],
 ['에', 'JKM'],
 ['도전', 'NNG'],
 ['하', 'XSV'],
 ['ㄴ', 'ETD'],
 ['배우', 'NNG'],
 ['이순', 'NNG'],
 ['재', 'NNG'],
 ['"', 'SS'],
 ['삶', 'NNG'],
 ['의', 'JKG'],


In [350]:
test_data['text'][2458]

'아르헨티나에서 한인 남성이 동포 여성을 살해한 뒤 음독을 시도했다가 경찰에 붙잡혔습니다.현지시간 13일 중남미 매체인 엘누에보닷컴과 시티오안디노 등에 따르면 이날 오전 아르헨티나 멘도사주에서 긴급 전화에 "독극물을 마셨다"는 신고가 접수됐습니다.아르헨티나 경찰은 신고자를 페루파토 병원으로 옮겼는데, 한국 국적의 김 모 씨로 밝혀진 이 남성은 경찰에게 지난주에 같은 국적의 아내를 죽였다"고 자백했습니다.경찰은 김 씨로부터 지난 9일쯤 함께 살던 동포 여성을 목 졸라 살해한 뒤 멘도사주 산마르틴 지역 돈페드로 농장 부근에 시신을 암매장했다는 추가 자백을 받아냈습니다.실제 김씨가 지목한 곳에서는 40대 여성으로 추정되는 시신이 발견된 것으로 파악됐습니다.김 씨는 현재 위중한 상태인 것으로 알려졌습니다.수사당국은 시신에 대해 부검을 하는 한편 법적 부부 여부 등 김 씨와 피해자 간 정확한 관계를 확인하고 있습니다.'

In [343]:
new_data, new_analysis = analysis(test_data)

100%|██████████| 3559/3559 [00:34<00:00, 104.56it/s]


In [349]:
all_analysis = get_more_features(new_data, new_analysis)

  0%|          | 0/3559 [00:00<?, ?it/s]

 69%|██████▉   | 2458/3559 [00:08<00:02, 465.16it/s]C:\Users\lhi30\AppData\Local\Temp\ipykernel_4584\3951035193.py:119: RuntimeWarning: invalid value encountered in divide
  token_variety = (num_token_type / num_token_sentences).tolist()
100%|██████████| 3559/3559 [00:10<00:00, 323.72it/s]


In [110]:
sentences = split_sentence(find_sent_id(tokens), tokens)
get_num_token_sentences(sentences)

[47, 39, 17, 45, 99, 32, 41, 34, 12, 22, 28, 10, 28, 33]

In [246]:
text = data['text'][1215]
text

"'스토킹 살인' 전주환 징역 40년 선고"

In [136]:
def tokenize(text: str) -> dict:
    kkma = Kkma()
    tokens = kkma.morphs(text)
    pos_tokens = kkma.pos(text)
    return {'text': text, 'tokens': tokens, 'pos': pos_tokens}

In [114]:
new_text

{'text': '본 연구는 소셜 빅데이터를 통해 국내에서 개봉된 일본 애니메이션 영화를 대상으로 흥행 요인을 분석하였다. 분석대상으로는 2017년 1월 국내에서 개봉한 너의 이름은.과 2021년 1월에 개봉한 귀멸의 칼날 무한열차편을 선정하였다. 분석 방법은 개봉 일자를 기준으로 유명 웹사이트에서 데이터를 수집하여, 추출된 키워드를 중심으로 텍스트 마이닝 기법인 의미연결망 분석과  분석, 감성 분석을 시행하였다. 분석 결과 두 작품은 소셜 미디어의 영향으로 온라인 구전 효과를 보았으며 관련 인물과 배급사, 제작 및 실사화, 이벤트, 상품성 등의 요인으로 화제가 되었다. 또한, 미디어믹스 성향이 강해 다양하게 소비할 수 있다는 강점도 드러났다. 이에 국내에서는 일본 애니메이션 영화를 소비하는 계층은 주로 블로그, 웹사이트, 를 통해 보는 경우가 많으며, 이 연구의 의의는 중심 키워드로부터 작품의 어떤 속성에 흥미가 있는지, 어떤 부분을 평가하고 있는지에 대해 분석했다는 점이다. ',
 'tokens': ['보',
  'ㄴ',
  '연구',
  '는',
  '소',
  '셜',
  '빅',
  '데이터',
  '를',
  '통하',
  '어',
  '국내',
  '에서',
  '개봉',
  '되',
  'ㄴ',
  '일본',
  '애니메이션',
  '영화',
  '를',
  '대상',
  '으로',
  '흥행',
  '요인',
  '을',
  '분석',
  '하',
  '였',
  '다',
  '.',
  '분석대상',
  '으로',
  '는',
  '2017',
  '년',
  '1',
  '월',
  '국내',
  '에서',
  '개봉',
  '하',
  'ㄴ',
  '너',
  '의',
  '이름',
  '은',
  '.',
  '과',
  '2021',
  '년',
  '1',
  '월',
  '에',
  '개봉',
  '하',
  'ㄴ',
  '귀멸',
  '의',
  '칼날',
  '무한',
  '열차',
  '편',
  '을',
  '선정',
  '

In [113]:
new_text = text_analysis(text)

In [133]:
def get_string_length(text: str) -> int:
    return(len(text))
def get_num_tokens(tokens: list) -> int:
    return(len(tokens))
def find_sent_id(tokens: list) -> list:
    ids = []
    for i, tok in enumerate(tokens):
        found = re.findall('[.?!]+', tok)
        if(bool(found)):
            ids.append(i)
    return ids
def split_sentence(sent_ids: list, tokens: list)-> list:  
    sentences = []
    start = 0
    for i in sent_ids:
        sentences.append(tokens[start:i+1])
        start = i+1
    return sentences
def split_pos_sentence(sent_ids: list, pos: list)-> list:  
    sentences = []
    start = 0
    for i in sent_ids:
        sent_both = pos[start:i+1] #get the sentence
        sent_tag = list(map(lambda tag: tag[1], sent_both))#get only the pos tags
        sentences.append(sent_tag)#append
        start = i+1
    return sentences
def get_num_sentences(tokens:list, pos:list):
    sent_ids = find_sent_id(tokens)
    sentences = split_sentence(sent_ids, tokens)
    pos_sentences = split_pos_sentence(sent_ids, pos)
    return len(sent_ids), sentences, pos_sentences
def get_num_token_sentences(sentences: list) -> list:
    num_tokens = []
    for sentence in sentences:
        num_tokens.append(len(sentence))
    return num_tokens

def simple_analysis(data):
    
    text = data['text']
    tokens = data['tokens']
    pos = data['pos']

    all_analysis = {}

    all_analysis['string_length'] = get_string_length(text)
    all_analysis['num_tokens'] = get_num_tokens(tokens)
    num_sentences, sentences, pos_sentences = get_num_sentences(tokens, pos)
    num_token_sentences = get_num_token_sentences(sentences)
    all_analysis['num_sentences'] = num_sentences
    all_analysis['num_token_sentences'] = num_token_sentences
    data['sentence_tokens'] = sentences
    data['sentence_pos'] = pos_sentences
    
    return data, all_analysis
def get_rel_freq(freq: dict):
    freq = pd.Series(freq, dtype = "float64")
    tot = freq.sum()
    return (freq/tot).to_dict()
def get_pos_freq(article: list[list]) -> dict:
    pos_freq = {}
    s_freq = []
    s_rel_freq = []
    article_freq = dict()

    for sentence in article:
        sentence_freq = dict(Counter(sentence))
        s_freq.append(sentence_freq)
        s_rel_freq.append(get_rel_freq(sentence_freq))
        for (key, freq) in sentence_freq.items():
            if key in article_freq.keys():
                article_freq[key] = article_freq[key] + freq
            else:
                article_freq[key] = freq

    pos_freq['s_freq'] = s_freq
    pos_freq['s_rel_freq'] = s_rel_freq
    pos_freq['a_freq'] = article_freq
    pos_freq['a_rel_freq'] = get_rel_freq(article_freq)
        
    return pos_freq 

def analysis(data):
    data, all_analysis = simple_analysis(data)
    all_analysis['pos_freq'] = get_pos_freq(data['sentence_pos'])
    return data, all_analysis

In [126]:
data, all_analysis = analysis(new_text)
print(data.keys())
print(all_analysis.keys())

dict_keys(['text', 'tokens', 'pos', 'sentence_tokens', 'sentence_pos'])
dict_keys(['string_length', 'num_tokens', 'num_sentences', 'num_token_sentences', 'pos_freq'])


In [129]:
example = {
    'text': text,
    'category': 'abstract'
    }
with open("C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/new_input/new_input.json", "w") as outfile:
    json.dump(example, outfile)

In [130]:
with open("C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/new_input/new_input.json") as json_file:
    data = json.load(json_file)
data

{'text': '본 연구는 소셜 빅데이터를 통해 국내에서 개봉된 일본 애니메이션 영화를 대상으로 흥행 요인을 분석하였다. 분석대상으로는 2017년 1월 국내에서 개봉한 너의 이름은.과 2021년 1월에 개봉한 귀멸의 칼날 무한열차편을 선정하였다. 분석 방법은 개봉 일자를 기준으로 유명 웹사이트에서 데이터를 수집하여, 추출된 키워드를 중심으로 텍스트 마이닝 기법인 의미연결망 분석과  분석, 감성 분석을 시행하였다. 분석 결과 두 작품은 소셜 미디어의 영향으로 온라인 구전 효과를 보았으며 관련 인물과 배급사, 제작 및 실사화, 이벤트, 상품성 등의 요인으로 화제가 되었다. 또한, 미디어믹스 성향이 강해 다양하게 소비할 수 있다는 강점도 드러났다. 이에 국내에서는 일본 애니메이션 영화를 소비하는 계층은 주로 블로그, 웹사이트, 를 통해 보는 경우가 많으며, 이 연구의 의의는 중심 키워드로부터 작품의 어떤 속성에 흥미가 있는지, 어떤 부분을 평가하고 있는지에 대해 분석했다는 점이다. ',
 'category': 'abstract'}

In [176]:
text = data['text']
tokenized = tokenize(text)
data, new_analysis = analysis(tokenized)

In [177]:
data.keys()

dict_keys(['text', 'tokens', 'pos', 'sentence_tokens', 'sentence_pos'])

In [154]:
pd.DataFrame(pd.Series(new_analysis['pos_freq']['a_rel_freq'])).T

,VV,ETD,NNG,JX,UN,JKO,ECS,JKM,XSV,EPT,...,ECE,JC,MAG,NNB,JKC,JKS,ECD,VXV,VA,MDT
0,0.044715,0.052846,0.373984,0.04065,0.012195,0.04878,0.02439,0.060976,0.052846,0.028455,...,0.012195,0.004065,0.01626,0.00813,0.004065,0.012195,0.004065,0.00813,0.004065,0.012195


In [188]:

# calculate number of tokens used / number of tokens in sentence
def calTokenVariety(new_analysis):
    num_token_type = np.array(list(map(len, new_analysis['pos_freq']['s_freq'])))
    num_token_sentences = np.array(new_analysis['num_token_sentences'])
    return num_token_type / num_token_sentences
# def calRelPosition(new_analysis, data):
#     sentence_pos = data['sentence_pos']
#     num_token_sentences = np.array(new_analysis['num_token_sentences'])
#     for i, sentence in enumerate(sentence_pos):
#         pos = np.array(sentence)
#         result = {}
#         for tag in set(pos):
#             indices = np.where(pos == tag)
#             result[tag] = indices/num_token_sentences[i]

#     return result
    

In [200]:
def calRelPosition(sentence_pos, num_token_sentences):
    for i, sentence in enumerate(sentence_pos):
        pos = np.array(sentence)
        result = {}
        for tag in set(pos):
            indices = np.where(pos == tag)
            result[tag] = (indices/num_token_sentences[i]).tolist()
            
    return result

In [185]:
a = np.array(data['sentence_pos'][0])
print(np.where(a == a[2]))
print(data['sentence_pos'][0])

(array([ 2,  4,  6,  7, 11, 13, 16, 17, 18, 20, 22, 23, 25], dtype=int64),)
['VV', 'ETD', 'NNG', 'JX', 'NNG', 'UN', 'NNG', 'NNG', 'JKO', 'VV', 'ECS', 'NNG', 'JKM', 'NNG', 'XSV', 'ETD', 'NNG', 'NNG', 'NNG', 'JKO', 'NNG', 'JKM', 'NNG', 'NNG', 'JKO', 'NNG', 'XSV', 'EPT', 'EFN', 'SF']


In [216]:
test_data = {
    'text': data['text'][:100],
    'tokens': data['tokens'][:100],
    'pos': data['pos'][:100]
    }
test_data['tokens'][10]

['YTN',
 '라디오',
 '(',
 'FM',
 '94.5',
 ')',
 'YTN',
 '\xa0',
 '뉴스',
 'FM',
 '\xa0',
 '슬기',
 '롭',
 'ㄴ',
 '라디오',
 '생활',
 '□',
 '\xa0',
 '방송',
 '일시',
 '\xa0',
 ':',
 '2023',
 '년',
 '2',
 '월',
 '20',
 '일',
 '(',
 '월요',
 '일',
 ')',
 '□',
 '\xa0',
 '진',
 '행',
 '\xa0',
 ':',
 '\xa0',
 '이',
 '현웅',
 '아나운서',
 '□',
 '\xa0',
 '출연',
 ':',
 '최',
 '영',
 '묵',
 '경기도',
 '가족',
 '다문화',
 '과',
 '과장',
 '*',
 '\xa0',
 '아래',
 '텍스트',
 '는',
 '실제',
 '방송',
 '내용',
 '과',
 '차이',
 '가',
 '있',
 '을',
 '수',
 '있',
 '으니',
 '보다',
 '정확',
 '하',
 'ㄴ',
 '내용',
 '은',
 '방송',
 '으로',
 '확인',
 '하',
 '시',
 '기',
 '바라',
 'ㅂ니다',
 '.',
 '◇',
 '\xa0',
 '이',
 '현웅',
 '아나운서',
 '(',
 '이하',
 '이',
 '현웅',
 ')',
 ':',
 '생활',
 '백서',
 ',',
 '월요일',
 '은',
 '경기도',
 '와',
 '함께',
 '하',
 'ㅂ니다',
 '.',
 '혼',
 '자',
 '있',
 '을',
 '때',
 '갑자기',
 '아프',
 '게',
 '되',
 '면',
 '병원',
 '갈',
 '는',
 '길',
 '도',
 '멀',
 '게',
 '느껴지',
 '고',
 '한편',
 '으로',
 '는',
 '서럽',
 'ㄴ',
 '마음',
 '까',
 '지',
 '도',
 '들',
 '곤',
 '하',
 '는데요',
 '.',
 '이렇게',
 '아프',
 'ㄹ',
 '때',
 '더',
 '힘들',
 'ㄴ',
 '1',


In [201]:
num_token_sentences = np.array(new_analysis['num_token_sentences'])
sentence_pos = data['sentence_pos']
calRelPosition(sentence_pos, num_token_sentences)


{'JKO': [[0.10294117647058823, 0.29411764705882354, 0.75]],
 'JX': [[0.04411764705882353, 0.17647058823529413, 0.5, 0.5588235294117647]],
 'EFN': [[0.9705882352941176]],
 'JKG': [[0.47058823529411764, 0.5882352941176471]],
 'SF': [[0.9852941176470589]],
 'XSV': [[0.1323529411764706, 0.7794117647058824, 0.8970588235294118]],
 'ECS': [[0.3235294117647059, 0.6911764705882353, 0.8676470588235294]],
 'MDT': [[0.4411764705882353, 0.6029411764705882, 0.7205882352941176]],
 'VXV': [[0.3382352941176471, 0.8088235294117647]],
 'SP': [[0.25, 0.27941176470588236, 0.4264705882352941, 0.7058823529411765]],
 'JKM': [[0.029411764705882353, 0.5441176470588235, 0.6323529411764706]],
 'ECE': [[0.4117647058823529, 0.7941176470588235]],
 'EPT': [[0.9117647058823529]],
 'VV': [[0.20588235294117646,
   0.3088235294117647,
   0.6764705882352942,
   0.8529411764705882]],
 'NNG': [[0.014705882352941176,
   0.058823529411764705,
   0.07352941176470588,
   0.08823529411764706,
   0.11764705882352941,
   0.1617647

In [190]:
calTokenVariety(new_analysis)

array([0.4       , 0.58823529, 0.57142857, 0.31111111, 0.41463415,
       0.58333333, 0.29411765])

[['음식', 'NNG'],
 ['을', 'JKO'],
 ['씹', 'VV'],
 ['기', 'ETN'],
 ['어렵', 'VA'],
 ['ㄴ', 'ETD'],
 ['노인', 'NNG'],
 ['은', 'JX'],
 ['노쇠', 'NNG'],
 ['위험', 'NNG'],
 ['이', 'JKS'],
 ['2.7', 'NR'],
 ['배', 'NNG'],
 ['높', 'VA'],
 ['아', 'ECD'],
 ['노년기', 'NNG'],
 ['에', 'JKM'],
 ['급격', 'XR'],
 ['하', 'XSA'],
 ['ㄴ', 'ETD'],
 ['노쇠', 'NNG'],
 ['를', 'JKO'],
 ['막', 'VV'],
 ['으려', 'ECD'],
 ['면', 'NNG'],
 ['평소', 'NNG'],
 ['치아', 'NNG'],
 ['건강', 'NNG'],
 ['을', 'JKO'],
 ['유지', 'NNG'],
 ['하', 'XSV'],
 ['는', 'ETD'],
 ['것', 'NNB'],
 ['이', 'JKS'],
 ['중요', 'NNG'],
 ['하', 'XSV'],
 ['ㄴ', 'ETD'],
 ['것', 'NNB'],
 ['으로', 'JKM'],
 ['나타나', 'VV'],
 ['었', 'EPT'],
 ['습니다', 'EFN'],
 ['.', 'SF'],
 ['서울', 'NNG'],
 ['아산', 'NNP'],
 ['병원', 'NNG'],
 ['노년', 'NNG'],
 ['내과', 'NNG'],
 ['정', 'NNG'],
 ['희원', 'NNG'],
 [',', 'SP'],
 ['빛', 'NNG'],
 ['고을', 'NNG'],
 ['전', 'NNG'],
 ['남대', 'NNG'],
 ['병원', 'NNG'],
 ['노년', 'NNG'],
 ['내과', 'NNG'],
 ['강', 'NNG'],
 ['민', 'NNG'],
 ['구', 'NNG'],
 ['교수', 'NNG'],
 ['공동', 'NNG'],
 ['연구', 'NNG'],
 ['팀', 'NNG'],

In [62]:
df_list = []
dic_list = all_analysis['pos_freq']['a_rel_freq']
for d in dic_list:
    df_list.append(pd.Series(d, dtype = "float64"))
all_df = pd.concat(df_list, axis = 1).T
all_df

,NNG,JKO,VV,ETD,JX,MAG,VA,EFN,SF,JKM,...,IC,EFA,JKI,EFO,MAC,OL,JKQ,SE,EPP,XPV
0,0.195440,0.028891,0.125620,0.049851,0.047869,0.043337,0.022093,0.062031,0.085824,0.032857,...,0.000850,0.000142,0.000142,0.000142,0.000283,NaN,NaN,NaN,NaN,NaN
1,0.194787,0.037723,0.127915,0.068244,0.036008,0.038752,0.022634,0.032236,0.041495,0.035665,...,NaN,0.000343,NaN,NaN,NaN,0.002401,0.000343,NaN,NaN,NaN
2,0.234636,0.036160,0.125993,0.056429,0.040701,0.046862,0.022864,0.048808,0.058537,0.047511,...,NaN,NaN,0.000162,0.000162,0.000162,0.000162,0.000162,NaN,NaN,NaN
3,0.238231,0.025963,0.108131,0.040799,0.061626,0.032240,0.013695,0.040799,0.055350,0.042225,...,0.001997,NaN,NaN,0.000285,0.000856,NaN,NaN,NaN,NaN,NaN
4,0.207165,0.036916,0.122890,0.051609,0.045780,0.037158,0.023072,0.052216,0.064117,0.038494,...,0.000486,0.000121,0.000243,0.000729,0.000729,NaN,0.000121,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.180974,0.037826,0.157418,0.041450,0.043262,0.048018,0.025595,0.041223,0.050057,0.027860,...,0.000680,0.000227,NaN,NaN,NaN,0.000680,NaN,NaN,NaN,NaN
104,0.157643,0.030255,0.112527,0.044055,0.045648,0.032909,0.027070,0.066879,0.079618,0.029724,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004246,NaN,NaN
105,0.200289,0.038410,0.106992,0.065961,0.048628,0.030867,0.022040,0.038464,0.040282,0.043546,...,NaN,0.000053,NaN,0.000053,0.000642,0.000214,0.000267,NaN,NaN,NaN
106,0.207422,0.031626,0.109200,0.068294,0.045427,0.030657,0.021864,0.038700,0.058338,0.036507,...,0.000308,0.000087,0.000274,0.000160,0.000582,0.000635,0.000127,NaN,0.000007,NaN


In [ ]:
def makeDF(dic_list):
    if type(dic_list) == dict:
        return pd.DataFrame(pd.Series(dic_list, dtype = "float64")).T             
    else:
        df_list = []
        for d in dic_list:
            df_list.append(pd.Series(d, dtype = "float64"))
        all_df = pd.concat(df_list, axis=1)
        return all_df.T

In [111]:
kkma_dict = {"NNG": "보통 명사", 
"NNP": "고유명사", 
"NNB": "일반 의존 명사",
"NNM": "단위 의존 명사",
"NR": "수사",
"NP": "대명사",
"VV": "동사",
"VA": "형용사",
"VXV": "보조 동사",
"VXA": "보조 형용사",
"VCP": "긍정 지정사, 서술격 조사 '이다'",
"VCN": "부정 지정사, 형용사 '아니다'",
"MDT": "일반 관형사",
"MDN": "수 관형사",
"MAG": "일반 부사",
"MAC": "접속 부사",
"IC": "감탄사",
"JKS": "주격 조사",
"JKC": "보격 조사",
"JKG": "관형격 조사",
"JKM": "부사격 조사",
"JKI": "호격 조사",
"JKQ": "인용격 조사",
"JX": "보조사",
"JC": "접속 조사",
"EPH": "존칭 선어말 어미",
"EPT": "시제 선어말 어미",
"EPP": "공손 선어말 어미",
"EFN": "평서형 종결 어미",
"EFQ": "의문형 종결 어미",
"EFO": "명령형 종결 어미",
"EFI": "감탄형 종결 어미",
"EFR": "존칭형 종결 어미",
"ECE": "대등 연결 어미",
"ECD": "의존적 연결 어미",
"ECS": "보조적 연결 어미",
"ETN": "명사형 전성 어미",
"ETD": "관형형 전성 어미",
"XPN": "체언 접두사",
"XPV": "용언 접두사",
"XSN": "명사 파생 접미사",
"XSV": "동사 파생 접미사",
"XSA": "형용사 파생 접미사",
"XR": "어근",
"SF": "마침표, 물음표, 느낌표",
"SP": "쉼표, 가운뎃점, 콜론, 빗금",
"SS": "따옴표, 괄호표, 줄표",
"SE": "줄임표",
"SO": "붙임표(물결, 숨김, 빠짐)",
"SW": "기타 기호(논리 수학 기호, 화폐 기호)",
"UN": "명사추정범주",
"OL": "외국어",
"OH": "한자",
"ON": "숫자"}

In [114]:
path = "C:/Users/lhi30/Haein/2023/YBIGTA/2023-2/DA/Writing_Advice/kkma_pos.json"
# path = input("Please enter the path to save the analysis result json file")
with open(path, "w") as outfile:
    json.dump(kkma_dict, outfile)

내가 해야 할 일:
- 새로운 토큰화 및 자료 분석 프로세스 만들기
변수 종류:
- 문장 당 사용한 token 종류 개수 / 문장의 token 수
